# Language Modelling in Hangman Game

This project implements an AI player for the classic Hangman word guessing game using character-level n-gram language models. The goal is to create automatic strategies that minimize incorrect guesses by learning patterns from text corpora.

## The Hangman Game Introduction

The Hangman game is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then. 

Here's a simple version of the game. 

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
    This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
    
    secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
    guesser: a function which guesses the next character at each stage in the game. The function takes a:
                - mask: what is known of the word, as a string with _ denoting an unknown character
                - guessed: the set of characters which already been guessed in the game
                - guesser_args: additional (optional) keyword arguments, i.e., name=value
    max_mistakes: limit on length of game, in terms of number of allowed mistakes
    verbose: silent or verbose diagnostic prints
    guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

In [4]:
interactive = False
if interactive:
    hangman('whatever', human, 8, True)

## Project Tasks

### 1. Data Preparation

This section prepares the dataset for training and testing the language models. We use words from NLTK's Brown corpus, filtering for alphabetic words and splitting into training and test sets.

In [5]:
from nltk.corpus import brown
import numpy as np

#nltk.download('brown')
np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

# Add all word types in Brown corpus into a words list
words_list = []

for word in brown.words():
    word = word.lower()
    if word.isalpha() and word not in words_list:
        words_list.append(word)

# Rando shuffle the words list and split it into a training set and a testing set
np.random.shuffle(words_list)

training_set = words_list[:-1000]
test_set = words_list[-1000:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


### 2. Language Models

#### 2.1 Baseline Random Guesser Model

This section implements a baseline random guessing strategy for comparison with more advanced language model-based approaches.


In [8]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [9]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###

    # Create a set that contains all letters
    chr_set = {"a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"}

    # Create a set of letters available to choose that is the difference set of all letters and guessed letters
    chr_available_list = list(chr_set - guessed)

    # Randomly choose a letter from list of available letters and return it
    chr_guess = np.random.choice(chr_available_list)

    return chr_guess

    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = status
Number of mistakes made by the random guesser = 20

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.777


#### 2.2 Unigram Language Model

This section implements a basic unigram language model that predicts characters based on their overall frequency in the training corpus.


In [11]:
unigram_counts = None

###
# Your answer BEGINS HERE
###

# Create an initial character count dictionary for reuse
INIT_DICT = {"a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,"j":0,"k":0,"l":0,"m":0,
             "n":0,"o":0,"p":0,"q":0,"r":0,"s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}

# Create a character count dictionary for uni-grams
unigram_counts = INIT_DICT.copy()

# Count the uni-gram characters in training set
for word in training_set:
    for chr in word:
        unigram_counts[chr] += 1

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###

    # Initialize the character with maximum uni-gram count
    max_chr = ""
    max_count = 0

    # Find the character with maximum uni-gram count and return it
    for chr in unigram_counts:
        if chr not in guessed and unigram_counts[chr] >= max_count:
            max_chr = chr
            max_count = unigram_counts[chr]

    return max_chr
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.808


#### 2.3 Unigram Length-Conditioned Model

This section improves the unigram model by conditioning character frequencies on word length, recognizing that different word lengths have different character distributions.


In [13]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###

# Create a dictionary to store the uni-gram count for different word length
unigram_counts_by_length = {}

# For each word length, create a uni-gram count dictionary
for word in training_set:

    word_len = len(word)

    if word_len in unigram_counts_by_length:
        for chr in word:
            unigram_counts_by_length[word_len][chr] += 1

    else:
        unigram_counts_by_length[word_len] = INIT_DICT.copy()
        for chr in word:
            unigram_counts_by_length[word_len][chr] += 1

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###

    # Obtain the uni-gram count dictionary of the length for word to be guessed;
    # If word length was never encountered in training set, use uni-gram count dictionary in question 3
    mask_len = len(mask)

    if mask_len in unigram_counts_by_length:
        unigram_counts_dict = unigram_counts_by_length[mask_len]
    else:
        unigram_counts_dict = unigram_counts

    # Find the character with maximum uni-gram count and return it
    max_chr = ""
    max_count = 0

    for chr in unigram_counts_dict:
        if chr not in guessed and unigram_counts_dict[chr] >= max_count:
            max_chr = chr
            max_count = unigram_counts_dict[chr]

    return max_chr

    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.706


#### 2.4 Bigram Language Model

This section implements a bigram language model that considers character sequences, using the previous character as context to predict the next one.

In [15]:
bigram_counts = None

###
# Your answer BEGINS HERE
###

# Create a revised initial character count dictionary for reuse (including start character)
INIT_DICT_REVISED_1 = {"$":0,
                       "a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,"j":0,"k":0,"l":0,"m":0,
                       "n":0,"o":0,"p":0,"q":0,"r":0,"s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}

# Part 1: Setup uni-gram count & uni-gram frequency
# Create a dictionary for uni-gram count
unigram_counts_revised = INIT_DICT_REVISED_1.copy()

for word in training_set:
    word = "$" + word
    for chr in word:
        unigram_counts_revised[chr] += 1

# Create a dictionary for uni-gram frequency
unigram_freq = INIT_DICT_REVISED_1.copy()

all_chr_count = sum(unigram_counts_revised.values())
for chr in unigram_freq:
    chr_count = unigram_counts_revised[chr]
    unigram_freq[chr] = chr_count / all_chr_count

# Part 2: Setup bi-gram count & bi-gram frequency
# Create a dictionary for bi-gram count
bigram_counts = {}

for word in training_set:
    word = "$" + word
    for i in range(len(word)-1):
        bi_1 = word[i]
        bi_2 = word[i+1]
        if bi_1 in bigram_counts:
            bigram_counts[bi_1][bi_2] += 1
        else:
            bigram_counts[bi_1] = INIT_DICT.copy()
            bigram_counts[bi_1][bi_2] += 1

# Create a dictionary for uni-gram frequency
bigram_freq = {}

for bi_1 in bigram_counts:
    bigram_freq[bi_1] = INIT_DICT.copy()
    bi_1_count = unigram_counts_revised[bi_1]
    for bi_2 in bigram_counts[bi_1]:
        bi_2_count = bigram_counts[bi_1][bi_2]
        bigram_freq[bi_1][bi_2] = bi_2_count / bi_1_count

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, bigram_freq=bigram_freq, unigram_freq=unigram_freq): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###

    # Revise the mask to add a start character at the beginning
    mask = ["$"] + mask

    # Create a set for all possible bi-gram combinations and a boolean flag for whether consider uni-gram frequency
    bigram_set = set()
    is_contain_unigram = False

    for i in range(len(mask)-1):
        bigram = bi_1, bi_2 = (mask[i], mask[i+1])
        if bi_1 != "_" and bi_2 == "_":
            bigram_set.add(bigram)
        if bi_1 == "_" and bi_2 == "_":
            is_contain_unigram = True

    # Sum up bi-gram frequencies and potentially uni-gram frequency across all letters (a-z)
    freq_dict = INIT_DICT.copy()

    for chr in freq_dict:
        freq = 0
        for bigram in bigram_set:
            bi_1 = bigram[0]
            freq += bigram_freq[bi_1][chr]
        if is_contain_unigram:
            freq += unigram_freq[chr]
        freq_dict[chr] = freq

    # Find the letter with maximum total frequency and return it
    max_chr = ""
    max_freq = 0

    for chr in freq_dict:
        if chr not in guessed and freq_dict[chr] >= max_freq:
            max_chr = chr
            max_freq = freq_dict[chr]

    return max_chr
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  9.118


#### 2.5 Self-Defined Advanced Language Model

This section explores more sophisticated approaches, including trigram models and other advanced techniques to improve guessing accuracy. The task is to define and implement your own advanced language model for Hangman, which could include techniques such as smoothing, backoff, or even neural network-based approaches.

The method I use is the combination of 3 tri-gram models plus a uni-gram model, where the tri-gram models consider all possible probability models of 3 characters (use 1 & 2 to predict 3; use 1 & 3 to predict 2; use 2 and 3 to predict 1). The probability estimations from 4 models are summed up as in Question 5. I use tri-gram models because combinations of 3 characters deliver more lexical information than combinations of 2 characters. Also, uni-gram model is retained, because for short words, individual characters deliver enough lexical information.

I have also tried the bidirectional bi-gram models, but the performance is not improved and even worse than not adding them. This also suggests that 3-character combination is more useful than 2-character combination.

In my method, the average number of incorrect guesses is 7.45 on the testing set, on average if I try it on different split of training & testing set. This is lower than all methods in previous questions, which suggests that the tri-gram model is more suitable for developing AI for the Hangman game.

In [16]:
###
# Your answer BEGINS HERE
###

## Part 0: Prepare initial character count dictionaries
# Create a revised initial character count dictionary for reuse (including start character)
INIT_DICT_REVISED_1 = {"$":0,
                       "a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,"j":0,"k":0,"l":0,"m":0,
                       "n":0,"o":0,"p":0,"q":0,"r":0,"s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}

# Create a revised initial character count dictionary for reuse (including end character)
INIT_DICT_REVISED_2 = {"&":0,
                       "a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,"j":0,"k":0,"l":0,"m":0,
                       "n":0,"o":0,"p":0,"q":0,"r":0,"s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}

# Create a revised initial character count dictionary for reuse (including both start and end character)
INIT_DICT_REVISED_3 = {"$":0,"&":0,
                       "a":0,"b":0,"c":0,"d":0,"e":0,"f":0,"g":0,"h":0,"i":0,"j":0,"k":0,"l":0,"m":0,
                       "n":0,"o":0,"p":0,"q":0,"r":0,"s":0,"t":0,"u":0,"v":0,"w":0,"x":0,"y":0,"z":0}

## Part 1: Setup uni-gram count & frequency
u_gram_counts = INIT_DICT_REVISED_3.copy()
for word in training_set:
    word = "$" + word + "&"
    for chr in word:
        u_gram_counts[chr] += 1

u_gram_freq = INIT_DICT_REVISED_3.copy()
for chr in u_gram_freq:
    u_gram_freq[chr] = u_gram_counts[chr] / sum(u_gram_counts.values())

# Part 2: Setup tri-gram count & tri-gram frequency in 3 ways
# Bi-gram count normal: two consecutive letters
# e.g., (a,b,c,d) >> ab, bc, cd
b_gram_count_normal = {}
for word in training_set:
    word = "$" + word + "&"
    for i in range(len(word)-1):
        bigram = word[i] + word[i+1]
        if bigram in b_gram_count_normal:
            b_gram_count_normal[bigram] += 1
        else:
            b_gram_count_normal[bigram] = 1

# Bi-gram count middle: two spaced letters
# e.g., (a,b,c,d) >> ac, bd
b_gram_count_middle = {}
for word in training_set:
    word = "$" + word + "&"
    for i in range(len(word)-2):
        bigram = word[i] + word[i+2]
        if bigram in b_gram_count_middle:
            b_gram_count_middle[bigram] += 1
        else:
            b_gram_count_middle[bigram] = 1

# Tri-gram: Forward
# e.g., (a,b,c,d) >> ab:c, bc:d
t_gram_counts_forwrd = {}
for word in training_set:
    word = "$" + word + "&"
    for i in range(len(word)-2):
        cnd = word[i] + word[i+1]
        obj = word[i+2]
        if cnd in t_gram_counts_forwrd:
            t_gram_counts_forwrd[cnd][obj] += 1
        else:
            t_gram_counts_forwrd[cnd] = INIT_DICT_REVISED_2.copy()
            t_gram_counts_forwrd[cnd][obj] += 1

t_gram_freq_forwrd = {}
for cnd in t_gram_counts_forwrd:
    t_gram_freq_forwrd[cnd] = INIT_DICT_REVISED_2.copy()
    for obj in t_gram_counts_forwrd[cnd]:
        t_gram_freq_forwrd[cnd][obj] = t_gram_counts_forwrd[cnd][obj] / b_gram_count_normal[cnd]

# Tri-gram: Backward
# e.g., (a,b,c,d) >> bc:a, cd:b
t_gram_counts_bckwrd = {}
for word in training_set:
    word = "$" + word + "&"
    for i in range(len(word)-2):
        cnd = word[i+1] + word[i+2]
        obj = word[i]
        if cnd in t_gram_counts_bckwrd:
            t_gram_counts_bckwrd[cnd][obj] += 1
        else:
            t_gram_counts_bckwrd[cnd] = INIT_DICT_REVISED_1.copy()
            t_gram_counts_bckwrd[cnd][obj] += 1

t_gram_freq_bckwrd = {}
for cnd in t_gram_counts_bckwrd:
    t_gram_freq_bckwrd[cnd] = INIT_DICT_REVISED_1.copy()
    for obj in t_gram_counts_bckwrd[cnd]:
        t_gram_freq_bckwrd[cnd][obj] = t_gram_counts_bckwrd[cnd][obj] / b_gram_count_normal[cnd]

# Tri-gram: Middle
# e.g., (a,b,c,d) >> ac:b, bd:c
t_gram_counts_middle = {}
for word in training_set:
    word = "$" + word + "&"
    for i in range(len(word)-2):
        cnd = word[i] + word[i+2]
        obj = word[i+1]
        if cnd in t_gram_counts_middle:
            t_gram_counts_middle[cnd][obj] += 1
        else:
            t_gram_counts_middle[cnd] = INIT_DICT_REVISED_3.copy()
            t_gram_counts_middle[cnd][obj] += 1

t_gram_freq_middle = {}
for cnd in t_gram_counts_middle:
    t_gram_freq_middle[cnd] = INIT_DICT_REVISED_3.copy()
    for obj in t_gram_counts_middle[cnd]:
        t_gram_freq_middle[cnd][obj] = t_gram_counts_middle[cnd][obj] / b_gram_count_middle[cnd]

## Part 3: Setup guesser
def my_amazing_ai_guesser(mask, guessed):

    # Revise the mask to add a start character at the beginning and an end character at the end
    mask = ["$"] + mask + ["&"]

    # Create a boolean flag for whether consider uni-gram frequency
    is_contain_unigram = False
    for i in range(len(mask)-1):
        bi_1, bi_2 = (mask[i], mask[i+1])
        if bi_1 == "_" and bi_2 == "_":
            is_contain_unigram = True

    # Create sets for all possible tri-gram combinations in three ways
    trigram_forwrd_set = set()
    trigram_bckwrd_set = set()
    trigram_middle_set = set()
    for i in range(len(mask)-2):
        trigram = ti_1, ti_2, ti_3 = (mask[i], mask[i+1], mask[i+2])
        if ti_1 != "_" and ti_2 != "_" and ti_3 == "_":
            trigram_forwrd_set.add(trigram)
        if ti_1 == "_" and ti_2 != "_" and ti_3 != "_":
            trigram_bckwrd_set.add(trigram)
        if ti_1 != "_" and ti_2 == "_" and ti_3 != "_":
            trigram_middle_set.add(trigram)

    # Sum up tri-gram frequencies in three ways and potentially uni-gram frequency across all letters (a-z)
    freq_dict = INIT_DICT.copy()
    for chr in freq_dict:
        freq = 0
        if is_contain_unigram:
            freq += u_gram_freq[chr]
        for trigram in trigram_forwrd_set:
            base = trigram[0] + trigram[1]
            freq += t_gram_freq_forwrd[base][chr]
        for trigram in trigram_bckwrd_set:
            base = trigram[1] + trigram[2]
            freq += t_gram_freq_bckwrd[base][chr]
        for trigram in trigram_middle_set:
            base = trigram[0] + trigram[2]
            freq += t_gram_freq_middle[base][chr]
        freq_dict[chr] = freq

    # Find the letter with maximum total frequency and return it
    max_chr = ""
    max_freq = 0
    for chr in freq_dict:
        if chr not in guessed and freq_dict[chr] >= max_freq:
            max_chr = chr
            max_freq = freq_dict[chr]

    return max_chr

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.421
